# Agrupando e manipulando dados agrupados com uso do Pandas

É muito comum em Data Science nós termos de responder perguntas como:
- 'Qual a média salarial **por sexo**?'
- 'Qual a média salarial **por região do Brasil**?'
- 'Qual a nota máxima do ENEM **por ano**?'


Entre muitas outras similares.

Vemos que todas essas perguntas possuem uma similaridade, que é a parte do final: 'por sexo', 'por cor', 'por região', 'por idade', 'por cargo', 'por ano'... 
Em geral, queremos avaliar uma métrica de acordo de acordo com cada **categoria** ou **grupo**.

Para responder esse tipo de pergunta, precisamos **agrupar** nossos dados de acordo com cada categoria.

O *pandas* possui uma funcionalidade muito bacana para nos ajudar, que é a função *groupby*.

Primeiro, importamos o pandas, nosso foco hoje, e o numpy, que possivelmente vai nos ajudar mais para frente.

In [1]:
import numpy as np
import pandas as pd

Antes de mais nada, as versões do numpy e pandas que utilizei para fazer esse notebook são, respectivamente, 1.19.2 e 1.2.3.

Isso é importante, pois bibliotecas costumam mudar conforme o tempo e algo que eu mostrar aqui pode ficar ultrapassado no futuro.

In [2]:
print(f'A versão do numpy é : {np.__version__}')
print(f'A versão do pandas é : {pd.__version__}')

A versão do numpy é : 1.19.2
A versão do pandas é : 1.2.3


Vamos agora importar nossa tabela.

Vou utilizar uma tabela do censo americado de 2010 a 2015.

In [3]:
df = pd.read_csv('census.csv')
df.head()

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


São 100 colunas ao todo, então vamos remover as que não nos interessam e manter apenas os nascimentos de cada ano e estimativa da população. Além disso, vou manter apenas as linhas onde o *sumarrary level* é *40*, pois essas são as linhas que se referem ao census de cada cidade.

In [4]:
colunas = ['STNAME', 'CTYNAME', 'BIRTHS2010', 'BIRTHS2011', 'BIRTHS2012', 'BIRTHS2013', 'BIRTHS2014',
          'BIRTHS2015', 'POPESTIMATE2010', 'POPESTIMATE2011', 'POPESTIMATE2012', 'POPESTIMATE2013', 
          'POPESTIMATE2014', 'POPESTIMATE2015']
df = df[df['SUMLEV'] == 50]
df = df[colunas]
df.head()

,STNAME,CTYNAME,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
1,Alabama,Autauga County,151,636,615,574,623,600,54660,55253,55175,55038,55290,55347
2,Alabama,Baldwin County,517,2187,2092,2160,2186,2240,183193,186659,190396,195126,199713,203709
3,Alabama,Barbour County,70,335,300,283,260,269,27341,27226,27159,26973,26815,26489
4,Alabama,Bibb County,44,266,245,259,247,253,22861,22733,22642,22512,22549,22583
5,Alabama,Blount County,183,744,710,646,618,603,57373,57711,57776,57734,57658,57673


# Formas de agrupar dados

Vamos agora entender como agrupar esses dados **por estado**. Suponha por exemplo, que queremos saber a média da população em cada estado, ou então a soma, ou mesmo os valores máximos e mínimos de cada cidade por estado.

Para começar a fazer essas análises, precisamos, antes de mais nada, agrupar essas 3142 linhas de acordo com os estados.

In [5]:
df['STNAME'].unique()

array(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California',
       'Colorado', 'Connecticut', 'Delaware', 'District of Columbia',
       'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana',
       'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland',
       'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi',
       'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire',
       'New Jersey', 'New Mexico', 'New York', 'North Carolina',
       'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania',
       'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee',
       'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'], dtype=object)

Aqui vemos que a coluna *STNAME* possui todos os estados dos EUA, mas que várias linhas são referente ao Alabama, várias linhas são referentes à Florida e assim por diante. A primeira forma que veremos de agrupar é justamente colocar no mesmo grupo aquelas linhas que têm o mesmo valor para uma determinada coluna.

In [6]:
# Utilizamos a função groupby passando o nome da coluna que queremos usar para agrupar
g = df.groupby('STNAME')

In [7]:
# Se tentarmos mostrar a variável g, vemos que ela é um objeto do tipo pandas.core.groupby.generic.DataFrameGroupBy
# Mas ainda não conseguimos entender o que realmente aconteceu com o nosso DataFrame
g

A variável `g` é um **objeto** como vários outro do *pandas* (como uma `Series` ou um `DataFrame`). Isso significa que temos várias funções e atributos dentro desse objeto que nós podemos usar.

Por exemplo, o método describe vai nos gerar várias informações, como em um DataFrame normal, mas de forma agrupada por estado.

In [8]:
g.describe().head(10)

BIRTHS2010                                             \
                          count         mean          std     min      25%   
STNAME                                                                       
Alabama                    67.0   212.328358   329.012866    21.0    43.00   
Alaska                     29.0    99.724138   229.620789     2.0     6.00   
Arizona                    15.0  1395.800000  3321.085498    18.0   173.00   
Arkansas                   75.0   125.853333   211.864228    14.0    34.00   
California                 58.0  2126.275862  4627.281089     2.0   112.75   
Colorado                   64.0   262.359375   561.172366     0.0    12.00   
Connecticut                 8.0  1186.125000  1081.829329   290.0   351.25   
Delaware                    3.0   949.000000   655.582947   569.0   570.50   
District of Columbia        1.0  2243.000000          NaN  2243.0  2243.00   
Florida                    67.0   759.283582  1293.232183    17.0    82.00   

                                               BIRTHS2011               ...  \
                         50%      75%      max      count         mean  ...   
STNAME                                                                  ...   
Alabama                102.0   233.50   2104.0       67.0   890.880597  ...   
Alaska                  22.0    56.00   1162.0       29.0   403.827586  ...   
Arizona                428.0   599.00  13123.0       15.0  5740.266667  ...   
Arkansas                53.0   109.50   1425.0       75.0   512.600000  ...   
California             571.5  2071.50  31740.0       58.0  8789.086207  ...   
Colorado                37.5   124.50   2405.0       64.0  1029.937500  ...   
Connecticut            537.0  2307.75   2719.0        8.0  4704.500000  ...   
Delaware               572.0  1139.00   1706.0        3.0  3766.333333  ...   
District of Columbia  2243.0  2243.00   2243.0        1.0  9196.000000  ...   
Florida                226.0   769.50   7275.0       67.0  3182.343284  ...   

                     POPESTIMATE2014             POPESTIMATE2015  \
                                 75%         max           count   
STNAME                                                             
Alabama                     81239.50    660368.0            67.0   
Alaska                      13781.00    300357.0            29.0   
Arizona                    211167.50   4090022.0            15.0   
Arkansas                    38663.50    392490.0            75.0   
California                 669323.25  10109436.0            58.0   
Colorado                    42219.25    663963.0            64.0   
Connecticut                870146.25    945816.0             8.0   
Delaware                   382097.50    553347.0             3.0   
District of Columbia       659836.00    659836.0             1.0   
Florida                    327242.50   2668903.0            67.0   

                                                                        \
                               mean           std       min        25%   
STNAME                                                                   
Alabama                72522.074627  1.058137e+05    8479.0   18163.00   
Alaska                 25463.172414  5.867997e+04     613.0    2534.00   
Arizona               455204.333333  1.056990e+06    9529.0   49810.00   
Arkansas               39709.386667  6.027020e+04    5229.0   12330.00   
California            674910.655172  1.477043e+06    1110.0   47048.25   
Colorado               85258.968750  1.731548e+05     701.0    5761.50   
Connecticut           448860.750000  3.778265e+05  116573.0  160902.25   
Delaware              315311.333333  2.101734e+05  173533.0  194577.50   
District of Columbia  672228.000000           NaN  672228.0  672228.00   
Florida               302556.298507  4.830767e+05    8331.0   27461.00   

                                                       
                           50%        75%         max  
STNAME            

Veja que, para cada coluna, temos várias informações. Por exemplo, temos a média, desvio padrão, máximo, mínimo, mediana para a população de 2015:

In [9]:
g.describe()['POPESTIMATE2015'].head(10)

,count,mean,std,min,25%,50%,75%,max
STNAME,,,,,,,,
Alabama,67.0,72522.074627,1.058137e+05,8479.0,18163.00,34123.0,81736.50,660367.0
Alaska,29.0,25463.172414,5.867997e+04,613.0,2534.00,6839.0,13709.00,298695.0
Arizona,15.0,455204.333333,1.056990e+06,9529.0,49810.00,126427.0,213496.00,4167947.0
Arkansas,75.0,39709.386667,6.027020e+04,5229.0,12330.00,18778.0,38683.00,392664.0
California,58.0,674910.655172,1.477043e+06,1110.0,47048.25,182321.5,679176.50,10170292.0
Colorado,64.0,85258.968750,1.731548e+05,701.0,5761.50,14336.5,42382.50,682545.0
Connecticut,8.0,448860.750000,3.778265e+05,116573.0,160902.25,227733.0,868562.75,948053.0
Delaware,3.0,315311.333333,2.101734e+05,173533.0,194577.50,215622.0,386200.50,556779.0
District of Columbia,1.0,672228.000000,NaN,672228.0,672228.00,672228.0,672228.00,672228.0


**Importante:** note que, depois de aplicarmos a função `describe`, o resultado é um `DataFrame`. Ou seja, podemos usar tudo aquilo que já sabemos sobre `DataFrame`'s (como acessar uma coluna, igual eu fiz acima).

In [10]:
type(g.describe())

pandas.core.frame.DataFrame

O `describe` nos traz muitas informações, mas podemos também aplicar outras funções para ter apenas aquilo que desejamos (e de forma mais rápida):

In [11]:
g.mean().head()

,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
STNAME,,,,,,,,,,,,
Alabama,212.328358,890.880597,881.522388,864.746269,870.656716,870.223881,71420.313433,71658.328358,71881.925373,72097.507463,72334.492537,72522.074627
Alaska,99.724138,403.827586,383.655172,391.310345,395.241379,396.482759,24621.413793,24921.379310,25214.758621,25429.034483,25415.379310,25463.172414
Arizona,1395.800000,5740.266667,5704.400000,5740.200000,5770.400000,5825.666667,427213.866667,431248.800000,436884.133333,442053.266667,448585.533333,455204.333333
Arkansas,125.853333,512.600000,514.720000,506.386667,505.493333,504.266667,38965.253333,39180.506667,39326.653333,39439.426667,39557.800000,39709.386667
California,2126.275862,8789.086207,8576.396552,8614.086207,8613.500000,8687.051724,643691.017241,650000.586207,656138.879310,662312.551724,668832.603448,674910.655172


In [12]:
g.max().head()

,CTYNAME,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
STNAME,,,,,,,,,,,,,
Alabama,Winston County,2104,8831,9007,8843,8621,8639,658116,657779,657704,658913,660368,660367
Alaska,Yukon-Koyukuk Census Area,1162,4783,4591,4698,4810,4796,293405,296333,298613,301537,300357,298695
Arizona,Yuma County,13123,53377,54292,54627,55060,55671,3825597,3871957,3945460,4015328,4090022,4167947
Arkansas,Yell County,1425,5800,5654,5656,5497,5514,383628,386827,388984,391352,392490,392664
California,Yuba County,31740,132649,129157,130647,129468,130070,9826009,9896602,9970436,10045175,10109436,10170292


In [13]:
g.count().head()

,CTYNAME,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
STNAME,,,,,,,,,,,,,
Alabama,67,67,67,67,67,67,67,67,67,67,67,67,67
Alaska,29,29,29,29,29,29,29,29,29,29,29,29,29
Arizona,15,15,15,15,15,15,15,15,15,15,15,15,15
Arkansas,75,75,75,75,75,75,75,75,75,75,75,75,75
California,58,58,58,58,58,58,58,58,58,58,58,58,58


A lista completa de operações pode ser encontrada [aqui](https://pandas.pydata.org/pandas-docs/stable/reference/groupby.html#computations-descriptive-stats).

# Acessando grupos

Em muitos casos, nós precisamos acessar um grupo específico ou mesmo iterar sobre nossos grupos. 

Para acessar um grupo específico, usamos a função `get_group` e passamos o grupo desejado.

In [14]:
g.get_group('California').head(10)

,STNAME,CTYNAME,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
191,California,Alameda County,4877,19219,18972,19466,19654,19996,1513754,1533052,1556952,1583845,1612850,1638215
192,California,Alpine County,2,6,6,6,7,4,1159,1111,1126,1144,1107,1110
193,California,Amador County,78,264,280,251,282,268,37873,37538,37099,36591,36744,37001
194,California,Butte County,623,2440,2397,2321,2501,2485,219977,220019,221205,222154,224033,225411
195,California,Calaveras County,74,333,356,323,348,349,45464,45101,44755,44574,44575,44828
196,California,Colusa County,70,336,302,314,312,316,21446,21389,21370,21389,21351,21482
197,California,Contra Costa County,3011,12335,11941,12245,12192,12534,1052921,1066636,1079290,1095959,1111710,1126745
198,California,Del Norte County,89,336,323,333,289,308,28566,28468,28201,27821,27194,27254
199,California,El Dorado County,382,1631,1598,1494,1553,1575,181148,180905,180579,181480,183050,184452
200,California,Fresno County,3893,16321,15950,15725,15954,16002,932462,940971,947713,955217,964983,974861


Note novamente que o resultado é um `DataFrame` onde todos os valores da coluna *STNAME* são 'California'.

O resultado é exatamente o mesmo que quando fazemos:

In [15]:
df[df['STNAME'] == 'California'].head(10)

,STNAME,CTYNAME,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
191,California,Alameda County,4877,19219,18972,19466,19654,19996,1513754,1533052,1556952,1583845,1612850,1638215
192,California,Alpine County,2,6,6,6,7,4,1159,1111,1126,1144,1107,1110
193,California,Amador County,78,264,280,251,282,268,37873,37538,37099,36591,36744,37001
194,California,Butte County,623,2440,2397,2321,2501,2485,219977,220019,221205,222154,224033,225411
195,California,Calaveras County,74,333,356,323,348,349,45464,45101,44755,44574,44575,44828
196,California,Colusa County,70,336,302,314,312,316,21446,21389,21370,21389,21351,21482
197,California,Contra Costa County,3011,12335,11941,12245,12192,12534,1052921,1066636,1079290,1095959,1111710,1126745
198,California,Del Norte County,89,336,323,333,289,308,28566,28468,28201,27821,27194,27254
199,California,El Dorado County,382,1631,1598,1494,1553,1575,181148,180905,180579,181480,183050,184452
200,California,Fresno County,3893,16321,15950,15725,15954,16002,932462,940971,947713,955217,964983,974861


A diferença é que `g` possui muitos outros grupos e é muito mais prático, como já vimos.

Para iterar pelos grupos, podemos usar um `for`.

In [16]:
# Para iterar é como se fosse um dicionário com a chave do grupo e o DataFrame
for chave, grupo in g:
    print(f'O nome do grupo é {chave} e o shape é {grupo.shape}.')

O nome do grupo é Alabama e o shape é (67, 14).
O nome do grupo é Alaska e o shape é (29, 14).
O nome do grupo é Arizona e o shape é (15, 14).
O nome do grupo é Arkansas e o shape é (75, 14).
O nome do grupo é California e o shape é (58, 14).
O nome do grupo é Colorado e o shape é (64, 14).
O nome do grupo é Connecticut e o shape é (8, 14).
O nome do grupo é Delaware e o shape é (3, 14).
O nome do grupo é District of Columbia e o shape é (1, 14).
O nome do grupo é Florida e o shape é (67, 14).
O nome do grupo é Georgia e o shape é (159, 14).
O nome do grupo é Hawaii e o shape é (5, 14).
O nome do grupo é Idaho e o shape é (44, 14).
O nome do grupo é Illinois e o shape é (102, 14).
O nome do grupo é Indiana e o shape é (92, 14).
O nome do grupo é Iowa e o shape é (99, 14).
O nome do grupo é Kansas e o shape é (105, 14).
O nome do grupo é Kentucky e o shape é (120, 14).
O nome do grupo é Louisiana e o shape é (64, 14).
O nome do grupo é Maine e o shape é (16, 14).
O nome do grupo é Maryl

Espero que agora esteja começando a ficar claro quais são as funcionalidade desse tipo de objeto.

Para terminar essa primeira parte, vale a pena dizer que podemos indexar colunas de um objeto `groupby`:

In [17]:
g[['BIRTHS2010','BIRTHS2011','BIRTHS2012','POPESTIMATE2014','POPESTIMATE2015']].mean().head()

,BIRTHS2010,BIRTHS2011,BIRTHS2012,POPESTIMATE2014,POPESTIMATE2015
STNAME,,,,,
Alabama,212.328358,890.880597,881.522388,72334.492537,72522.074627
Alaska,99.724138,403.827586,383.655172,25415.379310,25463.172414
Arizona,1395.800000,5740.266667,5704.400000,448585.533333,455204.333333
Arkansas,125.853333,512.600000,514.720000,39557.800000,39709.386667
California,2126.275862,8789.086207,8576.396552,668832.603448,674910.655172


# Outras formas de se agrupar

Agora que já entendemos alguns usos básicos de um agrupamento. Vamos voltar um pouco para a função `groupby` e ver rapidamente outras formas de utiliza-la.

### Várias colunas
A primeira forma é passando um array de colunas ao invés de uma única coluna.

In [18]:
df1 = pd.DataFrame({'col1' : [1, 1, 2, 2, 2, 3],
                  'col2' : ['no', 'yes', 'no', 'no', 'yes', 'no'],
                  'col_val1' : np.random.rand(6),
                  'col_val2' : np.random.randn(6)*10})
df1

,col1,col2,col_val1,col_val2
0,1,no,0.615946,2.180761
1,1,yes,0.744779,6.660194
2,2,no,0.045798,-1.630580
3,2,no,0.175219,16.988770
4,2,yes,0.710128,12.201738
5,3,no,0.334718,5.950345


In [19]:
df1.groupby(['col1', 'col2']).mean()

col_val1   col_val2
col1 col2                     
1    no    0.615946   2.180761
     yes   0.744779   6.660194
2    no    0.110509   7.679095
     yes   0.710128  12.201738
3    no    0.334718   5.950345

Perceba que agora temos um DataFrame com um MultiIndex onde o nível superior é 'col1' e o inferior é 'col2'. E note que não há uma linha `(3, 'yes')` porque essa combinação não existe.

### Por nível em um MultiIndex
Falando em MultiIndex, outra forma de agrupar é informando o nível em um MultiIndex.

In [20]:
colunas = pd.MultiIndex.from_arrays([['US']*3 +['JP']*2,
                                     [1, 3, 5, 1, 3]],
                                    names=['cty', 'num'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=colunas)
hier_df

cty        US                            JP          
num         1         3         5         1         3
0    1.020952  0.044413 -0.014104 -1.823596 -0.695255
1   -0.301043  0.707942  2.528698 -0.736551 -2.011227
2    0.352011 -1.079272 -0.491895 -0.190484  0.461092
3    0.930924  1.175761  0.082385 -0.786049 -0.641319

In [21]:
hier_df.groupby(level='cty', axis='columns').max()

cty,JP,US
0,-0.695255,1.020952
1,-0.736551,2.528698
2,0.461092,0.352011
3,-0.641319,1.175761


### Usando uma função

Podemos também agrupar utilizando uma função (criada por nós ou não). 

A função vai receber o índice de cada linha e deve devolver um valor que pode ser comparável. Então vamos criar os grupos de acordo com os resultados iguais da função.

In [22]:
df2 = pd.DataFrame({'nome' : ['João', 'Cláudia', 'Mariana', 'Mario', 'Joana', 'Lucas', 'Marcos'],
                   'nota1' : np.random.randint(0, 11, size=7),
                   'nota2' : np.random.randint(0, 11, size=7)})
df2 = df2.set_index('nome') # É importante que o índice seja os nomes, pois a função vai receber esses nomes
df2

,nota1,nota2
nome,,
João,10,5
Cláudia,8,4
Mariana,1,8
Mario,1,8
Joana,2,8
Lucas,0,10
Marcos,0,6


In [23]:
# Vamos criar uma função que pega uma string e pega suas primeiras duas letras
def duas_letras(s):
    # Sempre que tiver dúvida quanto a o que a função está recebendo, experimente printar o valor ou seu tipo
    #print(s)
    #print(type(s))
    return s[:2]
df2.groupby(duas_letras).max()

,nota1,nota2
Cl,8,4
Jo,10,8
Lu,0,10
Ma,1,8


Veja como nós agrupamos as pessoas de acordo com as duas primeiras letras de seu nome.

Poderíamos usar também funções do Python, como a função `len`:

In [24]:
df2.groupby(len).max()

,nota1,nota2
4,10,5
5,2,10
6,0,6
7,8,8


Se quisermos ver quem percente a que grupo, uma forma prática é utilizando o *atributo* `groups` do nosso objeto `groupby`.

In [25]:
df2.groupby(len).groups

{4: ['João'], 5: ['Mario', 'Joana', 'Lucas'], 6: ['Marcos'], 7: ['Cláudia', 'Mariana']}

# Agregações

Vamos agora passar para as principais aplicações e utilizades dos agrupamentos:
- Gerar agregações
- Gerar transformações
- Filtrar grupos

Começando com as agregações.

Sempre que temos uma lista e geramos um número com base naquela lista, estamos gerando uma agregação. Ou seja, uma agregação é sempre uma operação que transforma uma lista de valores em um único valor. Exemplos de agregação são soma, média, mediana, desvio padrão, máximo, mínimo, produto, entre outras agregações que nós mesmos podemos criar (agregações não se restringem a números).

Quando se trata de grupos, as agregações vão ser feitas dentro de cada grupo. Ou seja, se utilizamos uma média, vamos ter a média dos valores de cada grupo.

No pandas, as funções `agg` ou `aggregate` são utilizadas para fazer agregações (não existe diferença nenhuma entre elas, `agg` é um *alias* para *aggregate*).

In [26]:
# Vamos voltar ao nosso banco de dados do censo
g.agg(np.mean).head(10)

,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
STNAME,,,,,,,,,,,,
Alabama,212.328358,890.880597,881.522388,864.746269,870.656716,870.223881,71420.313433,71658.328358,71881.925373,72097.507463,72334.492537,72522.074627
Alaska,99.724138,403.827586,383.655172,391.310345,395.241379,396.482759,24621.413793,24921.379310,25214.758621,25429.034483,25415.379310,25463.172414
Arizona,1395.800000,5740.266667,5704.400000,5740.200000,5770.400000,5825.666667,427213.866667,431248.800000,436884.133333,442053.266667,448585.533333,455204.333333
Arkansas,125.853333,512.600000,514.720000,506.386667,505.493333,504.266667,38965.253333,39180.506667,39326.653333,39439.426667,39557.800000,39709.386667
California,2126.275862,8789.086207,8576.396552,8614.086207,8613.500000,8687.051724,643691.017241,650000.586207,656138.879310,662312.551724,668832.603448,674910.655172
Colorado,262.359375,1029.937500,1011.562500,1011.546875,1033.734375,1047.765625,78878.968750,79991.875000,81120.796875,82361.437500,83681.062500,85258.968750
Connecticut,1186.125000,4704.500000,4606.125000,4516.750000,4522.750000,4528.125000,447464.625000,448719.875000,449192.625000,449646.000000,449345.250000,448860.750000
Delaware,949.000000,3766.333333,3719.333333,3630.000000,3642.333333,3677.666667,299930.333333,302638.666667,305699.666667,308451.000000,311989.333333,315311.333333
District of Columbia,2243.000000,9196.000000,9234.000000,9447.000000,9415.000000,9593.000000,605126.000000,620472.000000,635342.000000,649540.000000,659836.000000,672228.000000


Note que isso é exatamente o que estávamos fazendo até agora:

In [27]:
g.mean().head(10)

,BIRTHS2010,BIRTHS2011,BIRTHS2012,BIRTHS2013,BIRTHS2014,BIRTHS2015,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015
STNAME,,,,,,,,,,,,
Alabama,212.328358,890.880597,881.522388,864.746269,870.656716,870.223881,71420.313433,71658.328358,71881.925373,72097.507463,72334.492537,72522.074627
Alaska,99.724138,403.827586,383.655172,391.310345,395.241379,396.482759,24621.413793,24921.379310,25214.758621,25429.034483,25415.379310,25463.172414
Arizona,1395.800000,5740.266667,5704.400000,5740.200000,5770.400000,5825.666667,427213.866667,431248.800000,436884.133333,442053.266667,448585.533333,455204.333333
Arkansas,125.853333,512.600000,514.720000,506.386667,505.493333,504.266667,38965.253333,39180.506667,39326.653333,39439.426667,39557.800000,39709.386667
California,2126.275862,8789.086207,8576.396552,8614.086207,8613.500000,8687.051724,643691.017241,650000.586207,656138.879310,662312.551724,668832.603448,674910.655172
Colorado,262.359375,1029.937500,1011.562500,1011.546875,1033.734375,1047.765625,78878.968750,79991.875000,81120.796875,82361.437500,83681.062500,85258.968750
Connecticut,1186.125000,4704.500000,4606.125000,4516.750000,4522.750000,4528.125000,447464.625000,448719.875000,449192.625000,449646.000000,449345.250000,448860.750000
Delaware,949.000000,3766.333333,3719.333333,3630.000000,3642.333333,3677.666667,299930.333333,302638.666667,305699.666667,308451.000000,311989.333333,315311.333333
District of Columbia,2243.000000,9196.000000,9234.000000,9447.000000,9415.000000,9593.000000,605126.000000,620472.000000,635342.000000,649540.000000,659836.000000,672228.000000


Mas então qual a diferença? Fazer apenas `.mean()` parece muito mais fácil.

A diferença é que a função `agg` nos permite fazer muitos mais. E é isso que vamos ver agora:

### Várias agregações
Com `agg` podemos passar quantas funções de agregação nós quisermos e vamos gerar um DataFrame com colunas hierarquicas com os nomes dessas funções:

In [28]:
g.agg([np.mean, np.sum]).head(10)

BIRTHS2010           BIRTHS2011           BIRTHS2012  \
                             mean     sum         mean     sum         mean   
STNAME                                                                        
Alabama                212.328358   14226   890.880597   59689   881.522388   
Alaska                  99.724138    2892   403.827586   11711   383.655172   
Arizona               1395.800000   20937  5740.266667   86104  5704.400000   
Arkansas               125.853333    9439   512.600000   38445   514.720000   
California            2126.275862  123324  8789.086207  509767  8576.396552   
Colorado               262.359375   16791  1029.937500   65916  1011.562500   
Connecticut           1186.125000    9489  4704.500000   37636  4606.125000   
Delaware               949.000000    2847  3766.333333   11299  3719.333333   
District of Columbia  2243.000000    2243  9196.000000    9196  9234.000000   
Florida                759.283582   50872  3182.343284  213217  3194.567164   

                               BIRTHS2013           BIRTHS2014          ...  \
                         sum         mean     sum         mean     sum  ...   
STNAME                                                                  ...   
Alabama                59062   864.746269   57938   870.656716   58334  ...   
Alaska                 11126   391.310345   11348   395.241379   11462  ...   
Arizona                85566  5740.200000   86103  5770.400000   86556  ...   
Arkansas               38604   506.386667   37979   505.493333   37912  ...   
California            497431  8614.086207  499617  8613.500000  499583  ...   
Colorado               64740  1011.546875   64739  1033.734375   66159  ...   
Connecticut            36849  4516.750000   36134  4522.750000   36182  ...   
Delaware               11158  3630.000000   10890  3642.333333   10927  ...   
District of Columbia    9234  9447.000000    9447  9415.000000    9415  ...   
Florida               214036  3179.164179  213004  3255.044776  218088  ...   

                     POPESTIMATE2011           POPESTIMATE2012            \
                                mean       sum            mean       sum   
STNAME                                                                     
Alabama                 71658.328358   4801108    71881.925373   4816089   
Alaska                  24921.379310    722720    25214.758621    731228   
Arizona                431248.800000   6468732   436884.133333   6553262   
Arkansas                39180.506667   2938538    39326.653333   2949499   
California             650000.586207  37700034   656138.879310  38056055   
Colorado                79991.875000   5119480    81120.796875   5191731   
Connecticut            448719.875000   3589759   449192.625000   3593541   
Delaware               302638.666667    907916   305699.666667    917099   
District of Columbia   620472.000000    620472   635342.000000    635342   
Florida                285157.208955  19105533   288836.134328  19352021   

                     POPESTIMATE2013           POPESTIMATE2014            \
                                mean       sum            mean       sum   
STNAME                                                                     
Alabama                 72097.507463   4830533    72334.492537   4846411   
Alaska                  25429.034483    737442    25415.379310    737046   
Arizona                442053.266667   6630799   448585.533333   6728783   
Arkansas                39439.426667   2957957    39557.800000   2966835   
California             662312.551724  38414128   668832.603448  38792291   
Colorado                82361.437500   5271132    83681.062500   5355588   
Connecticut            449646.000000   3597168   449345.250000   3594762   
Delaware               308451.000000    925353   311989.333333    935968   
District of Columbia   649540.000000    649540   659836.000000    659836   
Florida                292454.731343  19594467   297098.044776  19

### Agregações customizáveis
Além disso, podemos passar nossas próprias funções para a `agg`:

In [29]:
# O exemplo é besta, mas em muitos casos nós precisamos definir nossas próprias funções para gerar agregações 
# exatamente como desejamos

def meia_media(s):
    return s.mean() / 2
def soma_mais_10(s):
    return s.sum() + 10

g.agg([meia_media, soma_mais_10]).head(10)

BIRTHS2010                BIRTHS2011               \
                       meia_media soma_mais_10   meia_media soma_mais_10   
STNAME                                                                     
Alabama                106.164179        14236   445.440299        59699   
Alaska                  49.862069         2902   201.913793        11721   
Arizona                697.900000        20947  2870.133333        86114   
Arkansas                62.926667         9449   256.300000        38455   
California            1063.137931       123334  4394.543103       509777   
Colorado               131.179688        16801   514.968750        65926   
Connecticut            593.062500         9499  2352.250000        37646   
Delaware               474.500000         2857  1883.166667        11309   
District of Columbia  1121.500000         2253  4598.000000         9206   
Florida                379.641791        50882  1591.171642       213227   

                       BIRTHS2012                BIRTHS2013               \
                       meia_media soma_mais_10   meia_media soma_mais_10   
STNAME                                                                     
Alabama                440.761194        59072   432.373134        57948   
Alaska                 191.827586        11136   195.655172        11358   
Arizona               2852.200000        85576  2870.100000        86113   
Arkansas               257.360000        38614   253.193333        37989   
California            4288.198276       497441  4307.043103       499627   
Colorado               505.781250        64750   505.773438        64749   
Connecticut           2303.062500        36859  2258.375000        36144   
Delaware              1859.666667        11168  1815.000000        10900   
District of Columbia  4617.000000         9244  4723.500000         9457   
Florida               1597.283582       214046  1589.582090       213014   

                       BIRTHS2014               ... POPESTIMATE2011  \
                       meia_media soma_mais_10  ...      meia_media   
STNAME                                          ...                   
Alabama                435.328358        58344  ...    35829.164179   
Alaska                 197.620690        11472  ...    12460.689655   
Arizona               2885.200000        86566  ...   215624.400000   
Arkansas               252.746667        37922  ...    19590.253333   
California            4306.750000       499593  ...   325000.293103   
Colorado               516.867188        66169  ...    39995.937500   
Connecticut           2261.375000        36192  ...   224359.937500   
Delaware              1821.166667        10937  ...   151319.333333   
District of Columbia  4707.500000         9425  ...   310236.000000   
Florida               1627.522388       218098  ...   142578.604478   

                                  POPESTIMATE2012               \
                     soma_mais_10      meia_media soma_mais_10   
STNAME                                                           
Alabama                   4801118    35940.962687      4816099   
Alaska                     722730    12607.379310       731238   
Arizona                   6468742   218442.066667      6553272   
Arkansas                  2938548    19663.326667      2949509   
California               37700044   328069.439655     38056065   
Colorado                  5119490    40560.398438      5191741   
Connecticut               3589769   224596.312500      3593551   
Delaware                   907926   152849.833333       917109   
District of Columbia       620482   317671.000000       635352   
Florida                  19105543   144418.067164     19352031   

                     POPESTIMATE2013              POPESTIMATE2014  \
                          meia_media soma_mais_10      meia_media   
STNAME                                                              
Alabama                 36048.753731      4830543    36

Note que a função passada para `agg` recebe como parâmetro uma `Series` que corresponde a uma coluna do agrupamento. Vamos provar isso com `DataFrame` menor:

In [30]:
df2

,nota1,nota2
nome,,
João,10,5
Cláudia,8,4
Mariana,1,8
Mario,1,8
Joana,2,8
Lucas,0,10
Marcos,0,6


In [31]:
def verifica_tipo(s):
    print(type(s))
    print(s)
    print('-' * 100, end='\n\n')
    if s.shape[0] > 1:
        return 1000
    else:
        return 0
    
df2.groupby(duas_letras).agg(verifica_tipo)

<class 'pandas.core.series.Series'>
Cláudia    8
Name: nota1, dtype: int64
----------------------------------------------------------------------------------------------------

<class 'pandas.core.series.Series'>
João     10
Joana     2
Name: nota1, dtype: int64
----------------------------------------------------------------------------------------------------

<class 'pandas.core.series.Series'>
Lucas    0
Name: nota1, dtype: int64
----------------------------------------------------------------------------------------------------

<class 'pandas.core.series.Series'>
Mariana    1
Mario      1
Marcos     0
Name: nota1, dtype: int64
----------------------------------------------------------------------------------------------------

<class 'pandas.core.series.Series'>
Cláudia    4
Name: nota2, dtype: int64
----------------------------------------------------------------------------------------------------

<class 'pandas.core.series.Series'>
João     5
Joana    8
Name: nota2, dtype: in

,nota1,nota2
Cl,0,0
Jo,1000,1000
Lu,0,0
Ma,1000,1000


Veja como cada grupo aparece duas vezes, um para cada coluna do `DataFrame` original.

### Nomear agregações
Podemos, agora, querer renomear os nomes que aparecem em cada coluna da tabela agregada. Veja como podemos ter alguns casos em que renomar as colunas é bastante desejável:

In [32]:
def uma_funcao_com_um_nome_desnecessariamente_grande(s):
    return s.mean() / 2

g.agg([uma_funcao_com_um_nome_desnecessariamente_grande,
       lambda x : x.sum(),
       lambda x : x.max() - x.min()]).head(10)

BIRTHS2010  \
                     uma_funcao_com_um_nome_desnecessariamente_grande   
STNAME                                                                  
Alabama                                                    106.164179   
Alaska                                                      49.862069   
Arizona                                                    697.900000   
Arkansas                                                    62.926667   
California                                                1063.137931   
Colorado                                                   131.179688   
Connecticut                                                593.062500   
Delaware                                                   474.500000   
District of Columbia                                      1121.500000   
Florida                                                    379.641791   

                                            \
                     <lambda_0> <lambda_1>   
STNAME                                       
Alabama                   14226       2083   
Alaska                     2892       1160   
Arizona                   20937      13105   
Arkansas                   9439       1411   
California               123324      31738   
Colorado                  16791       2405   
Connecticut                9489       2429   
Delaware                   2847       1137   
District of Columbia       2243          0   
Florida                   50872       7258   

                                                           BIRTHS2011  \
                     uma_funcao_com_um_nome_desnecessariamente_grande   
STNAME                                                                  
Alabama                                                    445.440299   
Alaska                                                     201.913793   
Arizona                                                   2870.133333   
Arkansas                                                   256.300000   
California                                                4394.543103   
Colorado                                                   514.968750   
Connecticut                                               2352.250000   
Delaware                                                  1883.166667   
District of Columbia                                      4598.000000   
Florida                                                   1591.171642   

                                            \
                     <lambda_0> <lambda_1>   
STNAME                                       
Alabama                   59689       8736   
Alaska                    11711       4775   
Arizona                   86104      53263   
Arkansas                  38445       5759   
California               509767     132643   
Colorado                  65916       9584   
Connecticut               37636       9224   
Delaware                  11299       4673   
District of Columbia       9196          0   
Florida                  213217      31247   

                                                           BIRTHS2012  \
                     uma_funcao_com_um_nome_desnecessariamente_grande   
STNAME                                                                  
Alabama                                                    440.761194   
Alaska                                                     191.827586   
Arizona                                                   2852.200000   
Arkansas                                                   257.360000   
California                                                4288.198276   
Colorado                                                   505.781250   
Connecticut                                               2303.062500   
Delaware                                                  1859.666667   
District of Columbia                                      4617.000000   
Florida                                                   1597.283582   

           

Veja que temos uma função com um nome gigante e duas funções lambda (funções sem nome) que acabam não dando um resultado muito desejável. 

Podemos, entretanto, passar nomes para nossas colunas através da função `agg`. Ao invés de passar uma lista de funções, passamos uma lista de tuplas onde o primeiro elemento é o nome e o segundo a função:

In [33]:
g.agg([('meia_soma', uma_funcao_com_um_nome_desnecessariamente_grande),
       ('soma', lambda x : x.sum()),
       ('amplitude', lambda x : x.max() - x.min())]).head(10)

BIRTHS2010                     BIRTHS2011          \
                        meia_soma    soma amplitude    meia_soma    soma   
STNAME                                                                     
Alabama                106.164179   14226      2083   445.440299   59689   
Alaska                  49.862069    2892      1160   201.913793   11711   
Arizona                697.900000   20937     13105  2870.133333   86104   
Arkansas                62.926667    9439      1411   256.300000   38445   
California            1063.137931  123324     31738  4394.543103  509767   
Colorado               131.179688   16791      2405   514.968750   65916   
Connecticut            593.062500    9489      2429  2352.250000   37636   
Delaware               474.500000    2847      1137  1883.166667   11299   
District of Columbia  1121.500000    2243         0  4598.000000    9196   
Florida                379.641791   50872      7258  1591.171642  213217   

                                 BIRTHS2012                     BIRTHS2013  \
                     amplitude    meia_soma    soma amplitude    meia_soma   
STNAME                                                                       
Alabama                   8736   440.761194   59062      8902   432.373134   
Alaska                    4775   191.827586   11126      4584   195.655172   
Arizona                  53263  2852.200000   85566     54167  2870.100000   
Arkansas                  5759   257.360000   38604      5606   253.193333   
California              132643  4288.198276  497431    129151  4307.043103   
Colorado                  9584   505.781250   64740      9218   505.773438   
Connecticut               9224  2303.062500   36849      9070  2258.375000   
Delaware                  4673  1859.666667   11158      4590  1815.000000   
District of Columbia         0  4617.000000    9234         0  4723.500000   
Florida                  31247  1597.283582  214036     31065  1589.582090   

                      ... POPESTIMATE2012 POPESTIMATE2013                      \
                      ...       amplitude       meia_soma      soma amplitude   
STNAME                ...                                                       
Alabama               ...          648869    36048.753731   4830533    650207   
Alaska                ...          297960    12714.517241    737442    300901   
Arizona               ...         3936692   221026.633333   6630799   4006420   
Arkansas              ...          383678    19719.713333   2957957    386138   
California            ...         9969310   331156.275862  38414128  10044031   
Colorado              ...          644983    41180.718750   5271132    654451   
Connecticut           ...          817296   224823.000000   3597168    824207   
Delaware              ...          378600   154225.500000    925353    380144   
District of Columbia  ...               0   324770.000000    649540         0   
Florida               ...         2602939   146227.365672  19594467   2633620   

                     POPESTIMATE2014                     POPESTIMATE2015  \
                           meia_soma      soma amplitude       meia_soma   
STNAME                                                                     
Alabama                 36167.246269   4846411    651815    36261.037313   
Alaska                  12707.689655    737046    299723    12731.586207   
Arizona                224292.766667   6728783   4080699   227602.166667   
Arkansas                19778.900000   2966835    387302    19854.693333   
California             334416.301724  38792291  10108329   337455.327586   
Colorado                41840.531250   5355588    663268    42629.484375   
Connecticut            224672.625000   3594762    828986   224430.375000   
Delaware               155994.666667    935968    381574   157655.666667   
District of Columbia   329918.000000    659836         0   336114.000000   
Florida                148549.022388  19905569   2660

Existem outras formas de nomear, mas elas são muito mais complexas e eu acho que não vale a pena aprender.

Entretanto, recomendo você dar uma [olhada](https://pandas.pydata.org/docs/user_guide/groupby.html#named-aggregation).

### Agregações diferentes para colunas diferentes
Para finalizar as agregações, outra coisas que a função `agg` nos permite é aplicar agregações por coluna.

Note que eu passei três funções para `agg` e elas foram aplicadas em todas as colunas numéricas do meu `DataFrame`. Muitas vezes, não queremos isso. Pode ser que eu queira a soma dos nascimentos e a média e amplitude da quantidade de população (diferentes agregações para diferentes colunas). 

Podemos fazer isso passando um dicionário para `agg`:

In [34]:
minhas_agg = {'BIRTHS2013' : np.sum,
              'BIRTHS2014' : np.sum,
              'BIRTHS2015' : np.sum,
              'POPESTIMATE2013' : [np.mean, lambda x : x.max() - x.min()],
              'POPESTIMATE2014' : [np.mean, lambda x : x.max() - x.min()],
              'POPESTIMATE2015' : [np.mean, lambda x : x.max() - x.min()]}
g.agg(minhas_agg).head(10)

BIRTHS2013 BIRTHS2014 BIRTHS2015 POPESTIMATE2013  \
                            sum        sum        sum            mean   
STNAME                                                                  
Alabama                   57938      58334      58305    72097.507463   
Alaska                    11348      11462      11498    25429.034483   
Arizona                   86103      86556      87385   442053.266667   
Arkansas                  37979      37912      37820    39439.426667   
California               499617     499583     503849   662312.551724   
Colorado                  64739      66159      67057    82361.437500   
Connecticut               36134      36182      36225   449646.000000   
Delaware                  10890      10927      11033   308451.000000   
District of Columbia       9447       9415       9593   649540.000000   
Florida                  213004     218088     220628   292454.731343   

                                POPESTIMATE2014            POPESTIMATE2015  \
                     <lambda_0>            mean <lambda_0>            mean   
STNAME                                                                       
Alabama                  650207    72334.492537     651815    72522.074627   
Alaska                   300901    25415.379310     299723    25463.172414   
Arizona                 4006420   448585.533333    4080699   455204.333333   
Arkansas                 386138    39557.800000     387302    39709.386667   
California             10044031   668832.603448   10108329   674910.655172   
Colorado                 654451    83681.062500     663268    85258.968750   
Connecticut              824207   449345.250000     828986   448860.750000   
Delaware                 380144   311989.333333     381574   315311.333333   
District of Columbia          0   659836.000000          0   672228.000000   
Florida                 2633620   297098.044776    2660540   302556.298507   

                                 
                     <lambda_0>  
STNAME                           
Alabama                  651888  
Alaska                   298082  
Arizona                 4158418  
Arkansas                 387435  
California             10169182  
Colorado                 681844  
Connecticut              831480  
Delaware                 383246  
District of Columbia          0  
Florida                 2684786

Veja como obtivemos apenas as somas para os nascimentos e apenas a média e a amplitude para a quantidade de populão.

Além disso, perceba como apenas as colunas que eu informei apareceram (as colunas de 2010, 2011 e 2012 foram ignoradas).

Por fim, note que para ter mais de uma função por coluna, novamente usamos uma lista, como fazíamos antes.

Mas novamente tivemos um problema: a função lambda está sem nome. Será que você consegue resolver isso? 

**Exercício:** 
*Pegue tudo o que já aprendemos e crie uma agregação similar a anterior, mas com nomes 'soma', 'media' e 'amplitude'.*